In [ ]:
# Interactive Jupyter Notebook, running in Mercury for web application
#import warnings
#warnings.filterwarnings('ignore')
import plotly.express as px
import pandas as pd

import nltk
#nltk.download('punkt', quiet=True) # Frequency Distribution
#nltk.download('stopwords', quiet=True) # Stopwords for cleanup
#nltk.download('vader_lexicon', quiet=True) # Sentiment Analysis via VADER (Valence Aware Dictionary for Sentiment Reasoning)

In [ ]:
# Return the text file as tokens
user_defined_stopwords_to_remove = user_defined_stopwords.replace(" ", "").split(",")
with open(input_file, "r") as f:
	file_text = f.readlines()
file_text = " ".join(file_text) # combine all lines into one string
print(file_text)

# Break into Tokens and Clean-up
string_as_tokens_lst = nltk.word_tokenize(file_text)
print("Total Token Count: {0}".format(len(string_as_tokens_lst)))
stopwords = nltk.corpus.stopwords.words("english")
punctuation = ["--", "'", "''", "``", "?", "!", ".", ",", ";", ")", "(", "‘", "●", ":", '“', '”', '○', "[", "]", "&", '’', "%", "*", "–", "·", "-"]

# User defined stopwords
string_as_tokens_lst = [w for w in string_as_tokens_lst if w not in stopwords] # remove stopwords
string_as_tokens_lst = [w.lower() for w in string_as_tokens_lst if w not in punctuation] # remove punctuation
string_as_tokens_lst = [u for u in string_as_tokens_lst if u not in user_defined_stopwords_to_remove] # removes
print("Word Count (without words to be ignored): {0}\n".format(len(string_as_tokens_lst)))

In [ ]:
# Frequency Distribution
frequency_dist = nltk.FreqDist(string_as_tokens_lst)
frequencyDistribution_as_dict = dict(frequency_dist.most_common(most_freq_amount)) # convert to dict for plotting
print(frequencyDistribution_as_dict)
df = pd.DataFrame(frequencyDistribution_as_dict.items(), columns=["Word", "Occurence"])
df.set_index("Word",drop=True,inplace=True)
fig = px.bar(df, y="Occurence", title="Top Words")
fig.show()

In [ ]:
# Collocation Distribution
def plotNGram(n_gram_amount, n_gram_finder):
	# Plot N-Grams
	nGram_as_dict_temp = dict(n_gram_finder.ngram_fd.most_common(most_freq_amount)) # convert to dict for plotting

	nGram_as_dict = {}
	for k, v in nGram_as_dict_temp.items():
		nGram_as_dict[", ".join(k)] = v # rename key from ('graduate', 'division') to "graduate, division"
	df2 = pd.DataFrame(nGram_as_dict.items(), columns=["Word", "Occurence"])
	df2.set_index("Word",drop=True,inplace=True)
	fig = px.bar(df2, y="Occurence", title="Top Word {0}".format(n_gram_amount))
	fig.show()
	print(nGram_as_dict)    

In [ ]:
# Bigrams: Two-Word Combinations
bigram_collocation_dist = nltk.collocations.BigramCollocationFinder.from_words(string_as_tokens_lst)
plotNGram("Pairs", bigram_collocation_dist)

In [ ]:
# Trigrams: Three-Word Combinations
trigram_collocation_dist = nltk.collocations.TrigramCollocationFinder.from_words(string_as_tokens_lst)
plotNGram("Triplets", trigram_collocation_dist)

In [ ]:
# Quadgrams: Four-Word Combinations
quadgram_collocation_dist = nltk.collocations.QuadgramCollocationFinder.from_words(string_as_tokens_lst)
plotNGram("Quadruplets", quadgram_collocation_dist)